In [5]:
import pandas as pd

# Load data
file_path = './MagellanData.xlsx'
xls = pd.ExcelFile(file_path)
fmagx_df = xls.parse('FMAGX')
gacs_df = xls.parse('GACs')

# Clean FMAGX
fmagx_df.replace(-9999, pd.NA, inplace=True)
fmagx_df_long = fmagx_df.melt(id_vars=['yyyy'], var_name='month', value_name='return')
fmagx_df_long['month'] = fmagx_df_long['month'].str.extract('(\d+)').astype(int)
fmagx_df_long['date'] = pd.to_datetime(fmagx_df_long['yyyy'].astype(str) + '-' + fmagx_df_long['month'].astype(str))
fmagx_df_long = fmagx_df_long.drop(columns=['yyyy', 'month']).set_index('date').sort_index()
fmagx_df_long = fmagx_df_long[(fmagx_df_long['return'] > -100) & (fmagx_df_long['return'] < 100)]  # remove extreme outliers

# Clean GACs
gacs_df = gacs_df.rename(columns={'Month': 'date'})
gacs_df['date'] = pd.to_datetime(gacs_df['date'])
gacs_df.set_index('date', inplace=True)

# Export cleaned data
fmagx_df_long.to_csv('Cleaned_FMAGX.csv')
gacs_df.to_csv('Cleaned_GACs.csv')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load cleaned data
fmagx_df = pd.read_csv('Cleaned_FMAGX.csv', index_col='date', parse_dates=True)
gacs_df = pd.read_csv('Cleaned_GACs.csv', index_col='date', parse_dates=True)

# Visualize Monthly Returns of the Magellan Fund and S&P 500
plt.figure(figsize=(14, 6))
plt.plot(fmagx_df['return'], label='Magellan Fund', color='blue')
plt.plot(gacs_df['Core3'], label='S&P500 Core', color='orange', linestyle='--')
plt.title("Monthly Returns: Magellan Fund vs. S&P500")
plt.xlabel("Date")
plt.ylabel("Monthly Return (%)")
plt.legend()
plt.grid(True)
plt.show()

# Histogram of Monthly Returns for Magellan Fund
plt.figure(figsize=(10, 5))
plt.hist(fmagx_df['return'].dropna(), bins=50, color='blue', edgecolor='black', alpha=0.7)
plt.title("Distribution of Monthly Returns: Magellan Fund")
plt.xlabel("Monthly Return (%)")
plt.ylabel("Frequency")
plt.grid(axis='y')
plt.show()

# Optional: Rolling Mean and Volatility (e.g., 12-month rolling window)
fmagx_df['rolling_mean'] = fmagx_df['return'].rolling(window=12).mean()
fmagx_df['rolling_vol'] = fmagx_df['return'].rolling(window=12).std()

plt.figure(figsize=(14, 6))
plt.plot(fmagx_df['rolling_mean'], label='12-Month Rolling Mean', color='green')
plt.plot(fmagx_df['rolling_vol'], label='12-Month Rolling Volatility', color='red')
plt.title("12-Month Rolling Mean and Volatility of Magellan Fund Returns")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()


In [7]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.api import OLS

# Load cleaned data
fmagx_df = pd.read_csv('Cleaned_FMAGX.csv', index_col='date', parse_dates=True)
gacs_df = pd.read_csv('Cleaned_GACs.csv', index_col='date', parse_dates=True)

# Define manager periods
manager_periods = {
    'Peter Lynch': ('1980-02-01', '1990-05-01'),
    'Morris Smith': ('1990-06-01', '1992-06-01'),
    'Jeffrey Vinik': ('1992-07-01', '1996-05-01'),
    'Robert Stansky': ('1996-06-01', '2003-10-01')
}

# Benchmark returns (e.g., S&P 500 core from GACs)
benchmark_returns = gacs_df['Core3']

# Analysis Functions
def calculate_metrics(returns, benchmark):
    metrics = {}
    excess_returns = returns - benchmark.mean()
    metrics['mean_return'] = returns.mean()
    metrics['volatility'] = returns.std()
    metrics['sharpe_ratio'] = returns.mean() / returns.std()
    metrics['alpha'], metrics['beta'] = stats.linregress(benchmark, returns)[:2]
    return metrics



# Loop through managers
for manager, period in manager_periods.items():
    start, end = period
    returns = fmagx_df.loc[start:end, 'return']
    benchmark = benchmark_returns.loc[start:end]
    metrics = calculate_metrics(returns, benchmark)
    print(f"{manager} metrics:", metrics)


KeyError: 'Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is not allowed.'